In [56]:
import sys
import os

# os.getcwd() => '/home/genie/Documents/Projekti/cash-flow/jupyter'
project_root = os.path.abspath(os.path.join(os.getcwd(), '..'))  # Moves up one level to 'cash-flow'
# project_root = '/home/genie/Documents/Projekti/cash-flow'

# Add the module Converters directory to sys.path
sys.path.append(os.path.join(project_root, 'cash_flow', 'util'))


In [57]:
# Imports and engine

import pandas as pd
import numpy as np
import datetime
from sqlalchemy import create_engine

from Converters import date_format

engine = create_engine("sqlite:///../data/database.db", echo=False)

In [58]:
# Setup filters
date_from = "2024-01-12"
date_through = "2024-05-21"
# Choose period granularity: "day", "week", "month", "quarter", "year"
period = "month"
# Mapping period to offset (end of period)
period_offsets = {
    "day": pd.offsets.Day(0),
    "week": pd.offsets.Week(weekday=6),         # Sunday
    "month": pd.offsets.MonthEnd(0),
    "quarter": pd.offsets.QuarterEnd(startingMonth=12),
    "year": pd.offsets.YearEnd(0)
}
date_offset = period_offsets.get(period, pd.offsets.MonthEnd(0))
date_freq = {
    "day": "D",
    "week": "W-SUN",         # Sunday
    "month": "ME",
    "quarter": "QE",
    "year": "YE"
}
date_frequency = date_freq.get(period, "ME")

In [59]:
# Load all data from database

bank_df = pd.read_sql_query('SELECT * FROM G09_CashFlow_Actual WHERE d_date >= "' + date_from + '" AND d_date <= "' + date_through + '" ', engine)
cash_df = pd.read_sql_query('SELECT * FROM G02_CashTransactions WHERE d_date <= "' + date_through + '" ', engine)
definition_df = pd.read_sql_table("E01_CashFlowDefinition", engine)
definition_df.rename(columns={"id":"definition_id"}, inplace=True)
definition_acc_df = definition_df[definition_df["definition_type"] == 1]
definition_tot_df = definition_df[definition_df["definition_type"] == 2]
definition_bal_df = definition_df[definition_df["definition_type"] == 3]
definition_accounts_df = pd.read_sql_table("E01_CashFlowDefinitionAccounts", engine)
definition_totals_df = pd.read_sql_table("E01_CashFlowDefinitionTotals", engine)


In [60]:
bank_df.head()

,d_id,cash_status,cash_type,d_type,d_date,d_number,d_customer_id,d_vendor_ir,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC
0,2177,Actual,Receipt,1,2024-01-12 00:00:00.000000,1677,62.0,NaN,Payment 1677,EUR,DR,2620,1278.33,1278.33
1,2177,Actual,Receipt,1,2024-01-12 00:00:00.000000,1677,62.0,NaN,Payment 1677,EUR,CR,5721,221.86,221.86
2,2177,Actual,Receipt,1,2024-01-12 00:00:00.000000,1677,62.0,NaN,Payment 1677,EUR,CR,6550,1056.47,1056.47
3,2193,Actual,Receipt,1,2024-01-12 00:00:00.000000,1693,95.0,NaN,Payment 1693,EUR,DR,2620,4530.52,4530.52
4,2193,Actual,Receipt,1,2024-01-12 00:00:00.000000,1693,95.0,NaN,Payment 1693,EUR,CR,5721,786.29,786.29


In [61]:
cash_df.head()

,id,d_id,d_date,cash_type,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC
0,7780,2390,2020-01-01 00:00:00.000000,Receipt,EUR,DR,2620,1168.42,1168.42
1,7680,2340,2020-01-03 00:00:00.000000,Receipt,EUR,DR,2620,7758.84,7758.84
2,16215,4808,2020-01-04 00:00:00.000000,Payment,EUR,CR,2620,2912.90,2912.90
3,17083,5242,2020-01-06 00:00:00.000000,Payment,EUR,CR,2620,5131.03,5131.03
4,6128,1564,2020-01-09 00:00:00.000000,Receipt,EUR,DR,2620,5891.51,5891.51


In [62]:
definition_acc_df.head()

,definition_id,key,definition_type,name
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas
2,3,102,1.0,Maksājumi piegādātājiem
3,4,103,1.0,Maksājumi darbiniekiem
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi
6,7,201,1.0,Izdevumi procentu maksājumiem


In [63]:
definition_accounts_df.head()

,id,definition_id,cash_type,account
0,1,2,Receipt,6110
1,2,3,Payment,7110
2,3,4,Payment,7210
3,4,5,Receipt,6550
4,5,5,Payment,7510


In [64]:
definition_tot_df.head()

,definition_id,key,definition_type,name
5,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA
19,20,309,2.0,IEGULDĪJUMU DARBĪBAS NETO NAUDAS PLŪSMA
26,27,499,2.0,FINANSĒŠANAS DARBĪBAS NETO NAUDAS PLŪSMA
28,29,902,2.0,NETO NAUDAS PLŪSMA


In [65]:
definition_totals_df.head()

,id,definition_id,definition_summarized
0,1,6,2
1,2,6,3
2,3,6,4
3,4,6,5
4,5,10,2


In [66]:
definition_bal_df.head()

,definition_id,key,definition_type,name
29,30,999,3.0,NAUDAS LĪDZEKĻU ATLIKUMS PĀRSKATA PERIODA BEIGĀS


In [67]:
# ********************** (1) Start working with sums based on filtered accounts *******************************
# Merge accounts into accounts definition

definition_accounts_df = pd.merge(
    definition_acc_df,
    definition_accounts_df,
    on="definition_id", how="left")
definition_accounts_df.drop(columns=["id"], inplace=True)

In [68]:
definition_accounts_df

,definition_id,key,definition_type,name,cash_type,account
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,5721
2,3,102,1.0,Maksājumi piegādātājiem,Payment,7110
3,3,102,1.0,Maksājumi piegādātājiem,Payment,7310
4,3,102,1.0,Maksājumi piegādātājiem,Payment,5721
5,4,103,1.0,Maksājumi darbiniekiem,Payment,7210
6,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,6550
7,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,7510
8,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Receipt,5721
9,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,Payment,5721


In [69]:
# Merge accounts definition into bank transactions

merged_df = pd.merge(definition_accounts_df,
    bank_df,
    left_on=['cash_type', 'account'],        # Columns in definition_df
    right_on=['cash_type', 'gl_account'], # Corresponding columns in transactions_df
    how='left'
)


In [70]:
# Add column adjusted_amount_LC based on DR / CR

merged_df['adjusted_amount_LC'] = np.where(
    merged_df['gl_entry_type'] == 'CR', 
    merged_df['gl_amount_LC'], 
    -merged_df['gl_amount_LC']
)


In [71]:
# Convert d_date to_datetime if not done yet

merged_df['d_date'] = pd.to_datetime(merged_df['d_date'])

# Convert adjusted_amount_LC to_numeric if not done yet

merged_df['adjusted_amount_LC'] = pd.to_numeric(merged_df['adjusted_amount_LC'], errors='coerce')

In [72]:
# Add column for period reference

merged_df["d_period"] = merged_df['d_date'] + date_offset

In [73]:
merged_df.head()

,definition_id,key,definition_type,name,cash_type,account,d_id,cash_status,d_type,d_date,...,d_customer_id,d_vendor_ir,d_description,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC,adjusted_amount_LC,d_period
0,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2193.0,Actual,1.0,2024-01-12,...,95.0,NaN,Payment 1693,EUR,CR,6110,3744.23,3744.23,3744.23,2024-01-31
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2403.0,Actual,1.0,2024-01-12,...,77.0,NaN,Payment 1903,EUR,CR,6110,4098.39,4098.39,4098.39,2024-01-31
2,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2351.0,Actual,1.0,2024-01-16,...,35.0,NaN,Payment 1851,EUR,CR,6110,5668.21,5668.21,5668.21,2024-01-31
3,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,1590.0,Actual,1.0,2024-01-19,...,85.0,NaN,Payment 1090,EUR,CR,6110,426.89,426.89,426.89,2024-01-31
4,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,Receipt,6110,2265.0,Actual,1.0,2024-01-21,...,73.0,NaN,Payment 1765,EUR,CR,6110,5442.52,5442.52,5442.52,2024-01-31


In [74]:
merged_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 448 entries, 0 to 447
Data columns (total 21 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   definition_id       448 non-null    int64         
 1   key                 448 non-null    object        
 2   definition_type     448 non-null    float64       
 3   name                448 non-null    object        
 4   cash_type           431 non-null    object        
 5   account             431 non-null    object        
 6   d_id                431 non-null    float64       
 7   cash_status         431 non-null    object        
 8   d_type              431 non-null    float64       
 9   d_date              431 non-null    datetime64[ns]
 10  d_number            431 non-null    object        
 11  d_customer_id       189 non-null    float64       
 12  d_vendor_ir         242 non-null    float64       
 13  d_description       431 non-null    object        

In [75]:
# Pivot values based on definition_id and d_period

pivot_df = merged_df.pivot_table(
    index='definition_id',
    columns='d_period',
    values='adjusted_amount_LC',
    aggfunc="sum",
    fill_value=0,
    dropna=False
)


In [76]:
# Add all periods in range if some are missing

all_periods = pd.date_range(start=date_from, end=date_through, freq=date_frequency)
pivot_df = pivot_df.reindex(columns=all_periods, fill_value=0)


In [77]:
# Ensure columns are in datetime format before formatting
pivot_df.columns = pd.to_datetime(pivot_df.columns, errors='coerce')



In [78]:
pivot_df

,2024-01-31,2024-02-29,2024-03-31,2024-04-30
definition_id,,,,
2,36782.93,42880.14,26308.59,38379.57
3,-15262.82,-43778.47,-56641.43,-36223.79
4,-2715.78,-10580.35,-11534.17,-6987.47
5,29397.29,11550.30,-3411.93,6743.59
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00


In [79]:
# ********************** (2) Start working with totals based on totaled definitions *******************************
# Merge totals into totals definition
definition_totals_df = pd.merge(
    definition_tot_df,
    definition_totals_df,
    on="definition_id", how="left")
definition_totals_df.drop(columns=["id"], inplace=True)


In [80]:
definition_totals_df

,definition_id,key,definition_type,name,definition_summarized
0,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,2
1,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,3
2,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,4
3,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,5
4,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,2
5,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,3
6,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,4
7,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,5
8,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,7
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,8


In [81]:
# Merge totals definition with summarized accounts pivot

merged_totals_df = pd.merge(definition_totals_df,
    pivot_df,
    left_on='definition_summarized',
    right_on='definition_id', 
    how='left'
)

# Drop unnecessary columns

merged_totals_df.drop(columns = ['key', 'definition_type', 'name', 'definition_summarized'], inplace=True)


In [82]:
merged_totals_df

,definition_id,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00
0,6,36782.93,42880.14,26308.59,38379.57
1,6,-15262.82,-43778.47,-56641.43,-36223.79
2,6,-2715.78,-10580.35,-11534.17,-6987.47
3,6,29397.29,11550.30,-3411.93,6743.59
4,10,36782.93,42880.14,26308.59,38379.57
5,10,-15262.82,-43778.47,-56641.43,-36223.79
6,10,-2715.78,-10580.35,-11534.17,-6987.47
7,10,29397.29,11550.30,-3411.93,6743.59
8,10,0.00,0.00,0.00,0.00
9,10,0.00,0.00,0.00,0.00


In [83]:
# Choose value_columns for further summarization

value_columns = [col for col in merged_totals_df.columns if col not in ['definition_id']]

In [84]:
value_columns

[Timestamp('2024-01-31 00:00:00'),
 Timestamp('2024-02-29 00:00:00'),
 Timestamp('2024-03-31 00:00:00'),
 Timestamp('2024-04-30 00:00:00')]

In [108]:
# Summarize value_columns based on group value and op_num

summarized_totals_df = merged_totals_df.groupby('definition_id', group_keys=False)[value_columns].sum()

# .reset_index()

summarized_totals_df.fillna(0, inplace=True)

In [109]:
summarized_totals_df

,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00
definition_id,,,,
6,48201.62,71.62,-45278.94,1911.9
10,48201.62,71.62,-45278.94,1911.9
20,0.00,0.00,0.00,0.0
27,0.00,0.00,0.00,0.0
29,48201.62,71.62,-45278.94,1911.9


In [87]:
# ********************** (3) Start working with balances on end of each period *******************************

# Convert d_date to_datetime if not yet
cash_df['d_date'] = pd.to_datetime(cash_df['d_date'])


In [88]:
cash_df.head()

,id,d_id,d_date,cash_type,d_currency,gl_entry_type,gl_account,gl_amount,gl_amount_LC
0,7780,2390,2020-01-01,Receipt,EUR,DR,2620,1168.42,1168.42
1,7680,2340,2020-01-03,Receipt,EUR,DR,2620,7758.84,7758.84
2,16215,4808,2020-01-04,Payment,EUR,CR,2620,2912.90,2912.90
3,17083,5242,2020-01-06,Payment,EUR,CR,2620,5131.03,5131.03
4,6128,1564,2020-01-09,Receipt,EUR,DR,2620,5891.51,5891.51


In [89]:
cash_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype         
---  ------         --------------  -----         
 0   id             1460 non-null   int64         
 1   d_id           1460 non-null   int64         
 2   d_date         1460 non-null   datetime64[ns]
 3   cash_type      1460 non-null   object        
 4   d_currency     1460 non-null   object        
 5   gl_entry_type  1460 non-null   object        
 6   gl_account     1460 non-null   object        
 7   gl_amount      1460 non-null   float64       
 8   gl_amount_LC   1460 non-null   float64       
dtypes: datetime64[ns](1), float64(2), int64(2), object(4)
memory usage: 102.8+ KB


In [90]:
# Initialize cumulative balance
cumulative_balance = 0  
balances = []

# Start index for efficient filtering
start_idx = 0  

for eoperiod in value_columns:
    # Filter only new transactions from the last weekend up to the current one
    new_transactions = cash_df[(cash_df.index >= start_idx) & (cash_df['d_date'] <= eoperiod)]
    
    # If there are new transactions, update cumulative balance
    if not new_transactions.empty:
        cumulative_balance += new_transactions.apply(
            lambda row: row['gl_amount_LC'] if row['gl_entry_type'] == 'DR' else -row['gl_amount_LC'], axis=1
        ).sum()
        
        # Move the start index forward to avoid redundant calculations
        start_idx = new_transactions.index[-1] + 1  

    # Store balance for the current weekend
    balances.append(cumulative_balance)



In [91]:
# Create a DataFrame with periods as columns
balances_df = pd.DataFrame([balances], columns=value_columns)


In [92]:
balances_df

,2024-01-31,2024-02-29,2024-03-31,2024-04-30
0,195091.93,188989.78,179302.25,168630.03


In [93]:
# Create a new DataFrame with repeated rows for each definition_id
balances_df = pd.merge(definition_bal_df["definition_id"], balances_df, how='cross')
balances_df = balances_df.set_index("definition_id")


In [94]:
balances_df

,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00
definition_id,,,,
30,195091.93,188989.78,179302.25,168630.03


In [95]:
# ********************** (3) Put together summarized accounts, totals and balances on end of each period *******************************

In [96]:
pivot_df.head()

,2024-01-31,2024-02-29,2024-03-31,2024-04-30
definition_id,,,,
2,36782.93,42880.14,26308.59,38379.57
3,-15262.82,-43778.47,-56641.43,-36223.79
4,-2715.78,-10580.35,-11534.17,-6987.47
5,29397.29,11550.30,-3411.93,6743.59
7,0.00,0.00,0.00,0.00


In [97]:
summarized_totals_df.head()

,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00
definition_id,,,,
6,48201.62,71.62,-45278.94,1911.9
10,48201.62,71.62,-45278.94,1911.9
20,0.00,0.00,0.00,0.0
27,0.00,0.00,0.00,0.0
29,48201.62,71.62,-45278.94,1911.9


In [98]:
balances_df.head()

,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00
definition_id,,,,
30,195091.93,188989.78,179302.25,168630.03


In [99]:
# Concatenate it all together
report_df = pd.concat([pivot_df, summarized_totals_df, balances_df])


In [100]:
report_df

,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00
definition_id,,,,
2,36782.93,42880.14,26308.59,38379.57
3,-15262.82,-43778.47,-56641.43,-36223.79
4,-2715.78,-10580.35,-11534.17,-6987.47
5,29397.29,11550.30,-3411.93,6743.59
7,0.00,0.00,0.00,0.00
8,0.00,0.00,0.00,0.00
9,0.00,0.00,0.00,0.00
12,0.00,0.00,0.00,0.00
13,0.00,0.00,0.00,0.00


In [101]:
# ********************** (4) Prepare report for visual appearance *******************************

# Merge report definition header with report
report_df = pd.merge(definition_df, report_df, left_on="definition_id", right_on="definition_id", how="left")

# Sort based on key value
report_df.sort_values("key", inplace=True)


In [102]:
report_df

,definition_id,key,definition_type,name,2024-01-31 00:00:00,2024-02-29 00:00:00,2024-03-31 00:00:00,2024-04-30 00:00:00
0,1,100,NaN,PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN
1,2,101,1.0,Ieņēmumi no preču un pakalpojumu pārdošanas,36782.93,42880.14,26308.59,38379.57
2,3,102,1.0,Maksājumi piegādātājiem,-15262.82,-43778.47,-56641.43,-36223.79
3,4,103,1.0,Maksājumi darbiniekiem,-2715.78,-10580.35,-11534.17,-6987.47
4,5,104,1.0,Pārējie pamatdarbības ieņēmumi un izdevumi,29397.29,11550.30,-3411.93,6743.59
5,6,199,2.0,BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,48201.62,71.62,-45278.94,1911.90
6,7,201,1.0,Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00
7,8,202,1.0,Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00
8,9,203,1.0,Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00
9,10,299,2.0,PAMATDARBĪBAS NETO NAUDAS PLŪSMA,48201.62,71.62,-45278.94,1911.90


In [103]:
# Subtract report formatting in separate dataframe
format_df = report_df["definition_type"]

In [104]:

format_df

0     NaN
1     1.0
2     1.0
3     1.0
4     1.0
5     2.0
6     1.0
7     1.0
8     1.0
9     2.0
10    NaN
11    1.0
12    1.0
13    1.0
14    1.0
15    1.0
16    1.0
17    1.0
18    1.0
19    2.0
20    NaN
21    1.0
22    1.0
23    1.0
24    1.0
25    1.0
26    2.0
27    1.0
28    2.0
29    3.0
Name: definition_type, dtype: float64

In [105]:
# Prepare report_df for visual appearance

report_df.drop(columns=["definition_id", "key", "definition_type"], inplace=True)
report_df.set_index("name", inplace=True)

# Format column headers to show only the date part
report_df.columns = [col.strftime(date_format()) if not pd.isnull(col) else col for col in report_df.columns]



In [106]:
report_df

,"31. Jan, 2024","29. Feb, 2024","31. Mar, 2024","30. Apr, 2024"
name,,,,
PAMATDARBĪBAS NAUDAS PLŪSMA,NaN,NaN,NaN,NaN
Ieņēmumi no preču un pakalpojumu pārdošanas,36782.93,42880.14,26308.59,38379.57
Maksājumi piegādātājiem,-15262.82,-43778.47,-56641.43,-36223.79
Maksājumi darbiniekiem,-2715.78,-10580.35,-11534.17,-6987.47
Pārējie pamatdarbības ieņēmumi un izdevumi,29397.29,11550.30,-3411.93,6743.59
BRUTO PAMATDARBĪBAS NAUDAS PLŪSMA,48201.62,71.62,-45278.94,1911.90
Izdevumi procentu maksājumiem,0.00,0.00,0.00,0.00
Izdevumi nodokļu maksājumiem,0.00,0.00,0.00,0.00
Naudas plūsma no ārkārtas posteņiem,0.00,0.00,0.00,0.00


In [107]:
report_df.columns


Index(['31. Jan, 2024', '29. Feb, 2024', '31. Mar, 2024', '30. Apr, 2024'], dtype='object')